In [1]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

import re  #regular expression

In [2]:
sheets_to_load = ['Germany',
                  'Italy',
                  'Poland',
                  'Switzerland',
                  'Sweden',
                  'Thailand',
                  'UK',
                  'Vietnam',
                  'Australia']

# selected_sheets = pd.read_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\Compilation-raw\SEPTEMBER\q18-compilation.xlsx', sheet_name=sheets_to_load)
# selected_sheets = pd.read_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\Compilation-raw\OCTOBER\q18-compilation.xlsx', sheet_name=sheets_to_load)
# selected_sheets = pd.read_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\Compilation-raw\NOVEMBER\q18-compilation.xlsx', sheet_name=sheets_to_load)
selected_sheets = pd.read_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\Compilation-raw\DECEMBER\q18-compilation.xlsx', sheet_name=sheets_to_load)

Germany = selected_sheets['Germany']
Italy = selected_sheets['Italy']
Poland = selected_sheets['Poland']
Switzerland = selected_sheets['Switzerland']
Sweden = selected_sheets['Sweden']
Thailand = selected_sheets['Thailand']
UK = selected_sheets['UK']
Vietnam = selected_sheets['Vietnam']
Australia = selected_sheets['Australia']

### Germany

In [3]:
Germany.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396 entries, 0 to 395
Columns: 169 entries, Response ID to None of these:ROWENTA:GM - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.
dtypes: float64(168), int64(1)
memory usage: 523.0 KB


In [4]:
Germany.head(2) #initial dataset

,Response ID,"Refrigerator:BOSCH:GM - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Washing Machine:BOSCH:GM - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Tumble Dryers:BOSCH:GM - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Dishwashers:BOSCH:GM - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Vacuum Cleaners:BOSCH:GM - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Purifier:BOSCH:GM - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Conditioner:BOSCH:GM - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Oven:BOSCH:GM - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hob:BOSCH:GM - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.",...,"Vacuum Cleaners:ROWENTA:GM - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Purifier:ROWENTA:GM - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Conditioner:ROWENTA:GM - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Oven:ROWENTA:GM - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hob:ROWENTA:GM - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Cooker:ROWENTA:GM - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hood:ROWENTA:GM - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Microwave:ROWENTA:GM - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Fryers:ROWENTA:GM - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","None of these:ROWENTA:GM - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply."
0,78230,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,78263,NaN,NaN,NaN,NaN,NaN,6.0,NaN,8.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [5]:
# Unpivot the DataFrame, renaming 'Variable' to 'Product_Category'
unpivoted_GM = Germany.melt(id_vars=['Response ID'], var_name='Product_Category', value_name='Seen or Heard Y/N')

# Create a duplicate of the 'Product_Category' column named 'Product_Category_cleaned'
unpivoted_GM['Product_Category_cleaned'] = unpivoted_GM['Product_Category']

# Rearrange columns to place 'Product_Category_cleaned' next to 'Product_Category'
unpivoted_GM = unpivoted_GM[['Response ID', 'Product_Category', 'Product_Category_cleaned', 'Seen or Heard Y/N']]

# Remove the unwanted suffix from the 'Product_Category_cleaned' column only
unpivoted_GM['Product_Category_cleaned'] = unpivoted_GM['Product_Category_cleaned'].str.replace(
    ":GM - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.", 
    "", 
    regex=False
)

print(unpivoted_GM.tail())

       Response ID                                   Product_Category  \
66523        94276  None of these:ROWENTA:GM - Which product categ...   
66524        94287  None of these:ROWENTA:GM - Which product categ...   
66525        94291  None of these:ROWENTA:GM - Which product categ...   
66526        95452  None of these:ROWENTA:GM - Which product categ...   
66527        95582  None of these:ROWENTA:GM - Which product categ...   

      Product_Category_cleaned  Seen or Heard Y/N  
66523    None of these:ROWENTA                NaN  
66524    None of these:ROWENTA               99.0  
66525    None of these:ROWENTA                NaN  
66526    None of these:ROWENTA                NaN  
66527    None of these:ROWENTA                NaN  


In [6]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Product_Category_cleaned'
for entry in unpivoted_GM['Product_Category_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_GM['Brand'] = brand_names
unpivoted_GM['Ad_Type'] = ad_types

In [7]:
count = unpivoted_GM['Seen or Heard Y/N'].value_counts()
print(count)

Seen or Heard Y/N
2.0     491
5.0     491
1.0     377
99.0    317
8.0     298
10.0    293
4.0     288
3.0     277
9.0     221
12.0    191
13.0    187
11.0    166
6.0     144
7.0     125
Name: count, dtype: int64


In [8]:
# Drop the original 'Product_Category_cleaned' column
unpivoted_GM = unpivoted_GM.drop(columns=['Product_Category_cleaned'])

# Rename 'Ad_Type' to 'Product_Category_cleaned'
unpivoted_GM = unpivoted_GM.rename(columns={'Ad_Type': 'Product_Category_cleaned'})

# Rearrange columns to place 'Product_Category_cleaned' next to 'Product_Category'
unpivoted_GM = unpivoted_GM[['Response ID', 'Product_Category', 'Product_Category_cleaned', 'Seen or Heard Y/N', 'Brand']]

# Replace values in the 'Seen or Heard Y/N' column
unpivoted_GM['Seen or Heard Y/N'] = np.where(unpivoted_GM['Seen or Heard Y/N'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]), 1, 
                        np.where(unpivoted_GM['Seen or Heard Y/N'] == 99, 99,  
                        np.where(unpivoted_GM['Seen or Heard Y/N'].isna(), 0, unpivoted_GM['Seen or Heard Y/N'])))

In [9]:
unpivoted_GM.head()

,Response ID,Product_Category,Product_Category_cleaned,Seen or Heard Y/N,Brand
0,78230,Refrigerator:BOSCH:GM - Which product categori...,Refrigerator,0.0,BOSCH
1,78263,Refrigerator:BOSCH:GM - Which product categori...,Refrigerator,0.0,BOSCH
2,78268,Refrigerator:BOSCH:GM - Which product categori...,Refrigerator,0.0,BOSCH
3,78278,Refrigerator:BOSCH:GM - Which product categori...,Refrigerator,0.0,BOSCH
4,78299,Refrigerator:BOSCH:GM - Which product categori...,Refrigerator,0.0,BOSCH


In [10]:
count = unpivoted_GM['Seen or Heard Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_GM['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_GM)
print(f"Total number of rows: {total_rows}")

Seen or Heard Y/N
0.0     62662
1.0      3549
99.0      317
Name: count, dtype: int64


Brand
BOSCH        5544
MIELE        5544
SIEMENS      5544
AEG          5544
SAMSUNG      5544
BAUKNECHT    5544
PHILIPS      5544
LG           5544
DYSON        5544
NEFF         5544
HAIER        5544
ROWENTA      5544
Name: count, dtype: int64


Total number of rows: 66528


### Sweden

In [11]:
Sweden.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Columns: 169 entries, Response ID to None of these:ROBOROCK:SW - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.
dtypes: float64(168), int64(1)
memory usage: 528.3 KB


In [12]:
Sweden.head(2) #initial dataset

,Response ID,"Refrigerator:ELECTROLUX:SW - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Washing Machine:ELECTROLUX:SW - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Tumble Dryers:ELECTROLUX:SW - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Dishwashers:ELECTROLUX:SW - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Vacuum Cleaners:ELECTROLUX:SW - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Purifier:ELECTROLUX:SW - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Conditioner:ELECTROLUX:SW - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Oven:ELECTROLUX:SW - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hob:ELECTROLUX:SW - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.",...,"Vacuum Cleaners:ROBOROCK:SW - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Purifier:ROBOROCK:SW - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Conditioner:ROBOROCK:SW - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Oven:ROBOROCK:SW - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hob:ROBOROCK:SW - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Cooker:ROBOROCK:SW - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hood:ROBOROCK:SW - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Microwave:ROBOROCK:SW - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Fryers:ROBOROCK:SW - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","None of these:ROBOROCK:SW - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply."
0,78236,NaN,2.0,3.0,4.0,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,78238,1.0,2.0,NaN,4.0,NaN,NaN,NaN,8.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [13]:
# Unpivot the DataFrame, renaming 'Variable' to 'Product_Category'
unpivoted_SW = Sweden.melt(id_vars=['Response ID'], var_name='Product_Category', value_name='Seen or Heard Y/N')

# Create a duplicate of the 'Product_Category' column named 'Product_Category_cleaned'
unpivoted_SW['Product_Category_cleaned'] = unpivoted_SW['Product_Category']

# Rearrange columns to place 'Product_Category_cleaned' next to 'Product_Category'
unpivoted_SW = unpivoted_SW[['Response ID', 'Product_Category', 'Product_Category_cleaned', 'Seen or Heard Y/N']]

# Remove the unwanted suffix from the 'Product_Category_cleaned' column only
unpivoted_SW['Product_Category_cleaned'] = unpivoted_SW['Product_Category_cleaned'].str.replace(
    ":SW - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.", 
    "", 
    regex=False
)

print(unpivoted_SW.tail())

       Response ID                                   Product_Category  \
67195        95995  None of these:ROBOROCK:SW - Which product cate...   
67196        96005  None of these:ROBOROCK:SW - Which product cate...   
67197        96010  None of these:ROBOROCK:SW - Which product cate...   
67198        96062  None of these:ROBOROCK:SW - Which product cate...   
67199        96068  None of these:ROBOROCK:SW - Which product cate...   

      Product_Category_cleaned  Seen or Heard Y/N  
67195   None of these:ROBOROCK                NaN  
67196   None of these:ROBOROCK                NaN  
67197   None of these:ROBOROCK                NaN  
67198   None of these:ROBOROCK                NaN  
67199   None of these:ROBOROCK                NaN  


In [14]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Product_Category_cleaned'
for entry in unpivoted_SW['Product_Category_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_SW['Brand'] = brand_names
unpivoted_SW['Ad_Type'] = ad_types

In [15]:
count = unpivoted_SW['Seen or Heard Y/N'].value_counts()
print(count)

Seen or Heard Y/N
2.0     494
5.0     463
1.0     409
4.0     358
99.0    339
3.0     247
8.0     237
10.0    211
9.0     211
12.0    203
13.0    189
11.0    169
7.0     118
6.0      96
Name: count, dtype: int64


In [16]:
# Drop the original 'Product_Category_cleaned' column
unpivoted_SW = unpivoted_SW.drop(columns=['Product_Category_cleaned'])

# Rename 'Ad_Type' to 'Product_Category_cleaned'
unpivoted_SW = unpivoted_SW.rename(columns={'Ad_Type': 'Product_Category_cleaned'})

# Rearrange columns to place 'Product_Category_cleaned' next to 'Product_Category'
unpivoted_SW = unpivoted_SW[['Response ID', 'Product_Category', 'Product_Category_cleaned', 'Seen or Heard Y/N', 'Brand']]

# Replace values in the 'Seen or Heard Y/N' column
unpivoted_SW['Seen or Heard Y/N'] = np.where(unpivoted_SW['Seen or Heard Y/N'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]), 1, 
                        np.where(unpivoted_SW['Seen or Heard Y/N'] == 99, 99,  
                        np.where(unpivoted_SW['Seen or Heard Y/N'].isna(), 0, unpivoted_SW['Seen or Heard Y/N'])))

In [17]:
unpivoted_SW.head()

,Response ID,Product_Category,Product_Category_cleaned,Seen or Heard Y/N,Brand
0,78236,Refrigerator:ELECTROLUX:SW - Which product cat...,Refrigerator,0.0,ELECTROLUX
1,78238,Refrigerator:ELECTROLUX:SW - Which product cat...,Refrigerator,1.0,ELECTROLUX
2,78241,Refrigerator:ELECTROLUX:SW - Which product cat...,Refrigerator,1.0,ELECTROLUX
3,78246,Refrigerator:ELECTROLUX:SW - Which product cat...,Refrigerator,1.0,ELECTROLUX
4,78248,Refrigerator:ELECTROLUX:SW - Which product cat...,Refrigerator,0.0,ELECTROLUX


In [18]:
count = unpivoted_SW['Seen or Heard Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_SW['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_SW)
print(f"Total number of rows: {total_rows}")

Seen or Heard Y/N
0.0     63456
1.0      3405
99.0      339
Name: count, dtype: int64


Brand
ELECTROLUX    5600
BOSCH         5600
SAMSUNG       5600
MIELE         5600
PHILIPS       5600
SIEMENS       5600
CYLINDA       5600
WHIRLPOOL     5600
LG            5600
DYSON         5600
AEG           5600
ROBOROCK      5600
Name: count, dtype: int64


Total number of rows: 67200


### Italy

In [19]:
Italy.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Columns: 169 entries, Response ID to None of these:SIEMENS:IT - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.
dtypes: float64(168), int64(1)
memory usage: 525.6 KB


In [20]:
Italy.head(2) #initial dataset

,Response ID,"Refrigerator:SAMSUNG:IT - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Washing Machine:SAMSUNG:IT - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Tumble Dryers:SAMSUNG:IT - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Dishwashers:SAMSUNG:IT - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Vacuum Cleaners:SAMSUNG:IT - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Purifier:SAMSUNG:IT - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Conditioner:SAMSUNG:IT - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Oven:SAMSUNG:IT - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hob:SAMSUNG:IT - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.",...,"Vacuum Cleaners:SIEMENS:IT - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Purifier:SIEMENS:IT - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Conditioner:SIEMENS:IT - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Oven:SIEMENS:IT - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hob:SIEMENS:IT - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Cooker:SIEMENS:IT - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hood:SIEMENS:IT - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Microwave:SIEMENS:IT - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Fryers:SIEMENS:IT - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","None of these:SIEMENS:IT - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply."
0,78233,NaN,NaN,NaN,NaN,NaN,NaN,NaN,8.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,78237,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [21]:
# Unpivot the DataFrame, renaming 'Variable' to 'Product_Category'
unpivoted_IT = Italy.melt(id_vars=['Response ID'], var_name='Product_Category', value_name='Seen or Heard Y/N')

# Create a duplicate of the 'Product_Category' column named 'Product_Category_cleaned'
unpivoted_IT['Product_Category_cleaned'] = unpivoted_IT['Product_Category']

# Rearrange columns to place 'Product_Category_cleaned' next to 'Product_Category'
unpivoted_IT = unpivoted_IT[['Response ID', 'Product_Category', 'Product_Category_cleaned', 'Seen or Heard Y/N']]

# Remove the unwanted suffix from the 'Product_Category_cleaned' column only
unpivoted_IT['Product_Category_cleaned'] = unpivoted_IT['Product_Category_cleaned'].str.replace(
    ":IT - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.", 
    "", 
    regex=False
)

print(unpivoted_IT.tail())

       Response ID                                   Product_Category  \
66859        93279  None of these:SIEMENS:IT - Which product categ...   
66860        93284  None of these:SIEMENS:IT - Which product categ...   
66861        93290  None of these:SIEMENS:IT - Which product categ...   
66862        93341  None of these:SIEMENS:IT - Which product categ...   
66863        93357  None of these:SIEMENS:IT - Which product categ...   

      Product_Category_cleaned  Seen or Heard Y/N  
66859    None of these:SIEMENS                NaN  
66860    None of these:SIEMENS                NaN  
66861    None of these:SIEMENS                NaN  
66862    None of these:SIEMENS                NaN  
66863    None of these:SIEMENS                NaN  


In [22]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Product_Category_cleaned'
for entry in unpivoted_IT['Product_Category_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_IT['Brand'] = brand_names
unpivoted_IT['Ad_Type'] = ad_types

In [23]:
count = unpivoted_IT['Seen or Heard Y/N'].value_counts()
print(count)

Seen or Heard Y/N
2.0     801
1.0     602
4.0     539
3.0     492
5.0     444
8.0     428
12.0    313
9.0     308
7.0     300
10.0    259
99.0    252
6.0     223
11.0    174
13.0    128
Name: count, dtype: int64


In [24]:
# Drop the original 'Product_Category_cleaned' column
unpivoted_IT = unpivoted_IT.drop(columns=['Product_Category_cleaned'])

# Rename 'Ad_Type' to 'Product_Category_cleaned'
unpivoted_IT = unpivoted_IT.rename(columns={'Ad_Type': 'Product_Category_cleaned'})

# Rearrange columns to place 'Product_Category_cleaned' next to 'Product_Category'
unpivoted_IT = unpivoted_IT[['Response ID', 'Product_Category', 'Product_Category_cleaned', 'Seen or Heard Y/N', 'Brand']]

# Replace values in the 'Seen or Heard Y/N' column
unpivoted_IT['Seen or Heard Y/N'] = np.where(unpivoted_IT['Seen or Heard Y/N'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]), 1, 
                        np.where(unpivoted_IT['Seen or Heard Y/N'] == 99, 99,  
                        np.where(unpivoted_IT['Seen or Heard Y/N'].isna(), 0, unpivoted_IT['Seen or Heard Y/N'])))

In [25]:
unpivoted_IT.head()

,Response ID,Product_Category,Product_Category_cleaned,Seen or Heard Y/N,Brand
0,78233,Refrigerator:SAMSUNG:IT - Which product catego...,Refrigerator,0.0,SAMSUNG
1,78237,Refrigerator:SAMSUNG:IT - Which product catego...,Refrigerator,1.0,SAMSUNG
2,78242,Refrigerator:SAMSUNG:IT - Which product catego...,Refrigerator,0.0,SAMSUNG
3,78300,Refrigerator:SAMSUNG:IT - Which product catego...,Refrigerator,1.0,SAMSUNG
4,78315,Refrigerator:SAMSUNG:IT - Which product catego...,Refrigerator,0.0,SAMSUNG


In [26]:
count = unpivoted_IT['Seen or Heard Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_IT['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_IT)
print(f"Total number of rows: {total_rows}")

Seen or Heard Y/N
0.0     61601
1.0      5011
99.0      252
Name: count, dtype: int64


Brand
SAMSUNG       5572
BOSCH         5572
WHIRLPOOL     5572
LG            5572
DYSON         5572
MIELE         5572
ELECTROLUX    5572
HOTPOINT      5572
BEKO          5572
AEG           5572
HAIER         5572
SIEMENS       5572
Name: count, dtype: int64


Total number of rows: 66864


### UK

In [27]:
UK.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 396 entries, 0 to 395
Columns: 169 entries, Response ID to None of these:NEFF:UK - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.
dtypes: float64(168), int64(1)
memory usage: 523.0 KB


In [28]:
UK.head(2) #initial dataset

,Response ID,"Refrigerator:SAMSUNG:UK - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Washing Machine:SAMSUNG:UK - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Tumble Dryers:SAMSUNG:UK - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Dishwashers:SAMSUNG:UK - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Vacuum Cleaners:SAMSUNG:UK - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Purifier:SAMSUNG:UK - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Conditioner:SAMSUNG:UK - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Oven:SAMSUNG:UK - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hob:SAMSUNG:UK - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.",...,"Vacuum Cleaners:NEFF:UK - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Purifier:NEFF:UK - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Conditioner:NEFF:UK - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Oven:NEFF:UK - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hob:NEFF:UK - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Cooker:NEFF:UK - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hood:NEFF:UK - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Microwave:NEFF:UK - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Fryers:NEFF:UK - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","None of these:NEFF:UK - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply."
0,78251,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,78258,1.0,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [29]:
# Unpivot the DataFrame, renaming 'Variable' to 'Product_Category'
unpivoted_UK = UK.melt(id_vars=['Response ID'], var_name='Product_Category', value_name='Seen or Heard Y/N')

# Create a duplicate of the 'Product_Category' column named 'Product_Category_cleaned'
unpivoted_UK['Product_Category_cleaned'] = unpivoted_UK['Product_Category']

# Rearrange columns to place 'Product_Category_cleaned' next to 'Product_Category'
unpivoted_UK = unpivoted_UK[['Response ID', 'Product_Category', 'Product_Category_cleaned', 'Seen or Heard Y/N']]

# Remove the unwanted suffix from the 'Product_Category_cleaned' column only
unpivoted_UK['Product_Category_cleaned'] = unpivoted_UK['Product_Category_cleaned'].str.replace(
    ":UK - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.", 
    "", 
    regex=False
)

print(unpivoted_UK.tail())

       Response ID                                   Product_Category  \
66523        94064  None of these:NEFF:UK - Which product categori...   
66524        95433  None of these:NEFF:UK - Which product categori...   
66525        95591  None of these:NEFF:UK - Which product categori...   
66526        95639  None of these:NEFF:UK - Which product categori...   
66527        96098  None of these:NEFF:UK - Which product categori...   

      Product_Category_cleaned  Seen or Heard Y/N  
66523       None of these:NEFF                NaN  
66524       None of these:NEFF               99.0  
66525       None of these:NEFF                NaN  
66526       None of these:NEFF                NaN  
66527       None of these:NEFF                NaN  


In [30]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Product_Category_cleaned'
for entry in unpivoted_UK['Product_Category_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_UK['Brand'] = brand_names
unpivoted_UK['Ad_Type'] = ad_types

In [31]:
count = unpivoted_UK['Seen or Heard Y/N'].value_counts()
print(count)

Seen or Heard Y/N
2.0     513
5.0     462
1.0     414
10.0    313
4.0     301
3.0     293
8.0     278
12.0    243
99.0    233
7.0     205
9.0     192
13.0    181
6.0     169
11.0    158
Name: count, dtype: int64


In [32]:
# Drop the original 'Product_Category_cleaned' column
unpivoted_UK = unpivoted_UK.drop(columns=['Product_Category_cleaned'])

# Rename 'Ad_Type' to 'Product_Category_cleaned'
unpivoted_UK = unpivoted_UK.rename(columns={'Ad_Type': 'Product_Category_cleaned'})

# Rearrange columns to place 'Product_Category_cleaned' next to 'Product_Category'
unpivoted_UK = unpivoted_UK[['Response ID', 'Product_Category', 'Product_Category_cleaned', 'Seen or Heard Y/N', 'Brand']]

# Replace values in the 'Seen or Heard Y/N' column
unpivoted_UK['Seen or Heard Y/N'] = np.where(unpivoted_UK['Seen or Heard Y/N'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]), 1, 
                        np.where(unpivoted_UK['Seen or Heard Y/N'] == 99, 99,  
                        np.where(unpivoted_UK['Seen or Heard Y/N'].isna(), 0, unpivoted_UK['Seen or Heard Y/N'])))

In [33]:
unpivoted_UK.head()

,Response ID,Product_Category,Product_Category_cleaned,Seen or Heard Y/N,Brand
0,78251,Refrigerator:SAMSUNG:UK - Which product catego...,Refrigerator,0.0,SAMSUNG
1,78258,Refrigerator:SAMSUNG:UK - Which product catego...,Refrigerator,1.0,SAMSUNG
2,78259,Refrigerator:SAMSUNG:UK - Which product catego...,Refrigerator,1.0,SAMSUNG
3,78261,Refrigerator:SAMSUNG:UK - Which product catego...,Refrigerator,0.0,SAMSUNG
4,78262,Refrigerator:SAMSUNG:UK - Which product catego...,Refrigerator,0.0,SAMSUNG


In [34]:
count = unpivoted_UK['Seen or Heard Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_UK['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_UK)
print(f"Total number of rows: {total_rows}")

Seen or Heard Y/N
0.0     62573
1.0      3722
99.0      233
Name: count, dtype: int64


Brand
SAMSUNG     5544
LG          5544
HOTPOINT    5544
BOSCH       5544
DYSON       5544
BEKO        5544
SHARK       5544
ZANUSSI     5544
MIELE       5544
AEG         5544
SIEMENS     5544
NEFF        5544
Name: count, dtype: int64


Total number of rows: 66528


### Poland

In [35]:
Poland.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Columns: 169 entries, Response ID to None of these:AEG:PO - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.
dtypes: float64(168), int64(1)
memory usage: 528.3 KB


In [36]:
Poland.head(2) #initial dataset

,Response ID,"Refrigerator:SAMSUNG:PO - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Washing Machine:SAMSUNG:PO - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Tumble Dryers:SAMSUNG:PO - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Dishwashers:SAMSUNG:PO - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Vacuum Cleaners:SAMSUNG:PO - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Purifier:SAMSUNG:PO - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Conditioner:SAMSUNG:PO - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Oven:SAMSUNG:PO - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hob:SAMSUNG:PO - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.",...,"Vacuum Cleaners:AEG:PO - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Purifier:AEG:PO - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Conditioner:AEG:PO - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Oven:AEG:PO - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hob:AEG:PO - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Cooker:AEG:PO - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hood:AEG:PO - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Microwave:AEG:PO - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Fryers:AEG:PO - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","None of these:AEG:PO - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply."
0,78295,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,78297,NaN,2.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [37]:
# Unpivot the DataFrame, renaming 'Variable' to 'Product_Category'
unpivoted_PO = Poland.melt(id_vars=['Response ID'], var_name='Product_Category', value_name='Seen or Heard Y/N')

# Create a duplicate of the 'Product_Category' column named 'Product_Category_cleaned'
unpivoted_PO['Product_Category_cleaned'] = unpivoted_PO['Product_Category']

# Rearrange columns to place 'Product_Category_cleaned' next to 'Product_Category'
unpivoted_PO = unpivoted_PO[['Response ID', 'Product_Category', 'Product_Category_cleaned', 'Seen or Heard Y/N']]

# Remove the unwanted suffix from the 'Product_Category_cleaned' column only
unpivoted_PO['Product_Category_cleaned'] = unpivoted_PO['Product_Category_cleaned'].str.replace(
    ":PO - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.", 
    "", 
    regex=False
)

print(unpivoted_PO.tail())

       Response ID                                   Product_Category  \
67195        96000  None of these:AEG:PO - Which product categorie...   
67196        96002  None of these:AEG:PO - Which product categorie...   
67197        96003  None of these:AEG:PO - Which product categorie...   
67198        96016  None of these:AEG:PO - Which product categorie...   
67199        96017  None of these:AEG:PO - Which product categorie...   

      Product_Category_cleaned  Seen or Heard Y/N  
67195        None of these:AEG                NaN  
67196        None of these:AEG                NaN  
67197        None of these:AEG                NaN  
67198        None of these:AEG                NaN  
67199        None of these:AEG                NaN  


In [38]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Product_Category_cleaned'
for entry in unpivoted_PO['Product_Category_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_PO['Brand'] = brand_names
unpivoted_PO['Ad_Type'] = ad_types

In [39]:
count = unpivoted_PO['Seen or Heard Y/N'].value_counts()
print(count)

Seen or Heard Y/N
1.0     784
2.0     782
5.0     578
10.0    457
8.0     427
4.0     418
12.0    337
3.0     302
9.0     279
6.0     254
7.0     235
11.0    229
13.0    203
99.0    200
Name: count, dtype: int64


In [40]:
# Drop the original 'Product_Category_cleaned' column
unpivoted_PO = unpivoted_PO.drop(columns=['Product_Category_cleaned'])

# Rename 'Ad_Type' to 'Product_Category_cleaned'
unpivoted_PO = unpivoted_PO.rename(columns={'Ad_Type': 'Product_Category_cleaned'})

# Rearrange columns to place 'Product_Category_cleaned' next to 'Product_Category'
unpivoted_PO = unpivoted_PO[['Response ID', 'Product_Category', 'Product_Category_cleaned', 'Seen or Heard Y/N', 'Brand']]

# Replace values in the 'Seen or Heard Y/N' column
unpivoted_PO['Seen or Heard Y/N'] = np.where(unpivoted_PO['Seen or Heard Y/N'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]), 1, 
                        np.where(unpivoted_PO['Seen or Heard Y/N'] == 99, 99,  
                        np.where(unpivoted_PO['Seen or Heard Y/N'].isna(), 0, unpivoted_PO['Seen or Heard Y/N'])))

In [41]:
unpivoted_PO.head()

,Response ID,Product_Category,Product_Category_cleaned,Seen or Heard Y/N,Brand
0,78295,Refrigerator:SAMSUNG:PO - Which product catego...,Refrigerator,1.0,SAMSUNG
1,78297,Refrigerator:SAMSUNG:PO - Which product catego...,Refrigerator,0.0,SAMSUNG
2,78309,Refrigerator:SAMSUNG:PO - Which product catego...,Refrigerator,0.0,SAMSUNG
3,78317,Refrigerator:SAMSUNG:PO - Which product catego...,Refrigerator,1.0,SAMSUNG
4,78344,Refrigerator:SAMSUNG:PO - Which product catego...,Refrigerator,1.0,SAMSUNG


In [42]:
count = unpivoted_PO['Seen or Heard Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_PO['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_PO)
print(f"Total number of rows: {total_rows}")

Seen or Heard Y/N
0.0     61715
1.0      5285
99.0      200
Name: count, dtype: int64


Brand
SAMSUNG       5600
BOSCH         5600
ELECTROLUX    5600
BEKO          5600
AMICA         5600
PHILIPS       5600
WHIRLPOOL     5600
LG            5600
DYSON         5600
HAIER         5600
MIELE         5600
AEG           5600
Name: count, dtype: int64


Total number of rows: 67200


### Thailand

In [43]:
Thailand.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 399 entries, 0 to 398
Columns: 169 entries, Response ID to None of these:DYSON:TH - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.
dtypes: float64(168), int64(1)
memory usage: 526.9 KB


In [44]:
Thailand.head(2) #initial dataset

,Response ID,"Refrigerator:SAMSUNG:TH - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Washing Machine:SAMSUNG:TH - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Tumble Dryers:SAMSUNG:TH - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Dishwashers:SAMSUNG:TH - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Vacuum Cleaners:SAMSUNG:TH - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Purifier:SAMSUNG:TH - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Conditioner:SAMSUNG:TH - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Oven:SAMSUNG:TH - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hob:SAMSUNG:TH - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.",...,"Vacuum Cleaners:DYSON:TH - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Purifier:DYSON:TH - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Conditioner:DYSON:TH - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Oven:DYSON:TH - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hob:DYSON:TH - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Cooker:DYSON:TH - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hood:DYSON:TH - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Microwave:DYSON:TH - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Fryers:DYSON:TH - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","None of these:DYSON:TH - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply."
0,78252,NaN,NaN,NaN,NaN,NaN,6.0,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,78269,1.0,2.0,NaN,NaN,5.0,NaN,7.0,NaN,NaN,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
# Unpivot the DataFrame, renaming 'Variable' to 'Product_Category'
unpivoted_TH = Thailand.melt(id_vars=['Response ID'], var_name='Product_Category', value_name='Seen or Heard Y/N')

# Create a duplicate of the 'Product_Category' column named 'Product_Category_cleaned'
unpivoted_TH['Product_Category_cleaned'] = unpivoted_TH['Product_Category']

# Rearrange columns to place 'Product_Category_cleaned' next to 'Product_Category'
unpivoted_TH = unpivoted_TH[['Response ID', 'Product_Category', 'Product_Category_cleaned', 'Seen or Heard Y/N']]

# Remove the unwanted suffix from the 'Product_Category_cleaned' column only
unpivoted_TH['Product_Category_cleaned'] = unpivoted_TH['Product_Category_cleaned'].str.replace(
    ":TH - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.", 
    "", 
    regex=False
)

print(unpivoted_TH.tail())

       Response ID                                   Product_Category  \
67027        95265  None of these:DYSON:TH - Which product categor...   
67028        95350  None of these:DYSON:TH - Which product categor...   
67029        95488  None of these:DYSON:TH - Which product categor...   
67030        95492  None of these:DYSON:TH - Which product categor...   
67031        95815  None of these:DYSON:TH - Which product categor...   

      Product_Category_cleaned  Seen or Heard Y/N  
67027      None of these:DYSON                NaN  
67028      None of these:DYSON                NaN  
67029      None of these:DYSON                NaN  
67030      None of these:DYSON               99.0  
67031      None of these:DYSON                NaN  


In [46]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Product_Category_cleaned'
for entry in unpivoted_TH['Product_Category_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_TH['Brand'] = brand_names
unpivoted_TH['Ad_Type'] = ad_types

In [47]:
count = unpivoted_TH['Seen or Heard Y/N'].value_counts()
print(count)

Seen or Heard Y/N
1.0     1530
7.0     1492
2.0     1381
10.0    1046
5.0      986
12.0     957
6.0      956
13.0     777
3.0      764
11.0     694
8.0      686
4.0      589
9.0      571
99.0      97
Name: count, dtype: int64


In [48]:
# Drop the original 'Product_Category_cleaned' column
unpivoted_TH = unpivoted_TH.drop(columns=['Product_Category_cleaned'])

# Rename 'Ad_Type' to 'Product_Category_cleaned'
unpivoted_TH = unpivoted_TH.rename(columns={'Ad_Type': 'Product_Category_cleaned'})

# Rearrange columns to place 'Product_Category_cleaned' next to 'Product_Category'
unpivoted_TH = unpivoted_TH[['Response ID', 'Product_Category', 'Product_Category_cleaned', 'Seen or Heard Y/N', 'Brand']]

# Replace values in the 'Seen or Heard Y/N' column
unpivoted_TH['Seen or Heard Y/N'] = np.where(unpivoted_TH['Seen or Heard Y/N'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]), 1, 
                        np.where(unpivoted_TH['Seen or Heard Y/N'] == 99, 99,  
                        np.where(unpivoted_TH['Seen or Heard Y/N'].isna(), 0, unpivoted_TH['Seen or Heard Y/N'])))

In [49]:
unpivoted_TH.head()

,Response ID,Product_Category,Product_Category_cleaned,Seen or Heard Y/N,Brand
0,78252,Refrigerator:SAMSUNG:TH - Which product catego...,Refrigerator,0.0,SAMSUNG
1,78269,Refrigerator:SAMSUNG:TH - Which product catego...,Refrigerator,1.0,SAMSUNG
2,78313,Refrigerator:SAMSUNG:TH - Which product catego...,Refrigerator,0.0,SAMSUNG
3,78345,Refrigerator:SAMSUNG:TH - Which product catego...,Refrigerator,1.0,SAMSUNG
4,78363,Refrigerator:SAMSUNG:TH - Which product catego...,Refrigerator,0.0,SAMSUNG


In [50]:
count = unpivoted_TH['Seen or Heard Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_TH['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_TH)
print(f"Total number of rows: {total_rows}")

Seen or Heard Y/N
0.0     54506
1.0     12429
99.0       97
Name: count, dtype: int64


Brand
SAMSUNG       5586
LG            5586
MITSUBISHI    5586
ELECTROLUX    5586
PANASONIC     5586
SHARP         5586
PHILIPS       5586
HITACHI       5586
HAIER         5586
TEFAL         5586
BOSCH         5586
DYSON         5586
Name: count, dtype: int64


Total number of rows: 67032


### Vietnam

In [51]:
Vietnam.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 400 entries, 0 to 399
Columns: 169 entries, Response ID to None of these:MIDEA:VN - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.
dtypes: float64(168), int64(1)
memory usage: 528.3 KB


In [52]:
Vietnam.head(2) #initial dataset

,Response ID,"Refrigerator:SAMSUNG:VN - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Washing Machine:SAMSUNG:VN - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Tumble Dryers:SAMSUNG:VN - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Dishwashers:SAMSUNG:VN - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Vacuum Cleaners:SAMSUNG:VN - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Purifier:SAMSUNG:VN - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Conditioner:SAMSUNG:VN - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Oven:SAMSUNG:VN - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hob:SAMSUNG:VN - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.",...,"Vacuum Cleaners:MIDEA:VN - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Purifier:MIDEA:VN - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Conditioner:MIDEA:VN - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Oven:MIDEA:VN - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hob:MIDEA:VN - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Cooker:MIDEA:VN - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hood:MIDEA:VN - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Microwave:MIDEA:VN - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Fryers:MIDEA:VN - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","None of these:MIDEA:VN - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply."
0,78271,NaN,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,78275,1.0,2.0,NaN,NaN,5.0,6.0,7.0,8.0,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [53]:
# Unpivot the DataFrame, renaming 'Variable' to 'Product_Category'
unpivoted_VN = Vietnam.melt(id_vars=['Response ID'], var_name='Product_Category', value_name='Seen or Heard Y/N')

# Create a duplicate of the 'Product_Category' column named 'Product_Category_cleaned'
unpivoted_VN['Product_Category_cleaned'] = unpivoted_VN['Product_Category']

# Rearrange columns to place 'Product_Category_cleaned' next to 'Product_Category'
unpivoted_VN = unpivoted_VN[['Response ID', 'Product_Category', 'Product_Category_cleaned', 'Seen or Heard Y/N']]

# Remove the unwanted suffix from the 'Product_Category_cleaned' column only
unpivoted_VN['Product_Category_cleaned'] = unpivoted_VN['Product_Category_cleaned'].str.replace(
    ":VN - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.", 
    "", 
    regex=False
)

print(unpivoted_VN.tail())

       Response ID                                   Product_Category  \
67195        95842  None of these:MIDEA:VN - Which product categor...   
67196        95844  None of these:MIDEA:VN - Which product categor...   
67197        95845  None of these:MIDEA:VN - Which product categor...   
67198        95848  None of these:MIDEA:VN - Which product categor...   
67199        95852  None of these:MIDEA:VN - Which product categor...   

      Product_Category_cleaned  Seen or Heard Y/N  
67195      None of these:MIDEA                NaN  
67196      None of these:MIDEA                NaN  
67197      None of these:MIDEA                NaN  
67198      None of these:MIDEA                NaN  
67199      None of these:MIDEA                NaN  


In [54]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Product_Category_cleaned'
for entry in unpivoted_VN['Product_Category_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_VN['Brand'] = brand_names
unpivoted_VN['Ad_Type'] = ad_types

In [55]:
count = unpivoted_VN['Seen or Heard Y/N'].value_counts()
print(count)

Seen or Heard Y/N
1.0     1228
2.0     1134
7.0     1122
6.0      813
10.0     800
5.0      746
8.0      692
12.0     669
13.0     643
9.0      617
4.0      594
3.0      592
11.0     577
99.0      41
Name: count, dtype: int64


In [56]:
# Drop the original 'Product_Category_cleaned' column
unpivoted_VN = unpivoted_VN.drop(columns=['Product_Category_cleaned'])

# Rename 'Ad_Type' to 'Product_Category_cleaned'
unpivoted_VN = unpivoted_VN.rename(columns={'Ad_Type': 'Product_Category_cleaned'})

# Rearrange columns to place 'Product_Category_cleaned' next to 'Product_Category'
unpivoted_VN = unpivoted_VN[['Response ID', 'Product_Category', 'Product_Category_cleaned', 'Seen or Heard Y/N', 'Brand']]

# Replace values in the 'Seen or Heard Y/N' column
unpivoted_VN['Seen or Heard Y/N'] = np.where(unpivoted_VN['Seen or Heard Y/N'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]), 1, 
                        np.where(unpivoted_VN['Seen or Heard Y/N'] == 99, 99,  
                        np.where(unpivoted_VN['Seen or Heard Y/N'].isna(), 0, unpivoted_VN['Seen or Heard Y/N'])))

In [57]:
unpivoted_VN.head()

,Response ID,Product_Category,Product_Category_cleaned,Seen or Heard Y/N,Brand
0,78271,Refrigerator:SAMSUNG:VN - Which product catego...,Refrigerator,0.0,SAMSUNG
1,78275,Refrigerator:SAMSUNG:VN - Which product catego...,Refrigerator,1.0,SAMSUNG
2,78287,Refrigerator:SAMSUNG:VN - Which product catego...,Refrigerator,1.0,SAMSUNG
3,78291,Refrigerator:SAMSUNG:VN - Which product catego...,Refrigerator,0.0,SAMSUNG
4,78358,Refrigerator:SAMSUNG:VN - Which product catego...,Refrigerator,1.0,SAMSUNG


In [58]:
count = unpivoted_VN['Seen or Heard Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_VN['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_VN)
print(f"Total number of rows: {total_rows}")

Seen or Heard Y/N
0.0     56932
1.0     10227
99.0       41
Name: count, dtype: int64


Brand
SAMSUNG       5600
PANASONIC     5600
LG            5600
TOSHIBA       5600
ELECTROLUX    5600
SHARP         5600
PHILIPS       5600
HITACHI       5600
XIAOMI        5600
AQUA          5600
BOSCH         5600
MIDEA         5600
Name: count, dtype: int64


Total number of rows: 67200


### Australia

In [59]:
Australia.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 397 entries, 0 to 396
Columns: 169 entries, Response ID to None of these:AEG:AU - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.
dtypes: float64(168), int64(1)
memory usage: 524.3 KB


In [60]:
Australia.head(2) #initial dataset

,Response ID,"Refrigerator:SAMSUNG:AU - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Washing Machine:SAMSUNG:AU - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Tumble Dryers:SAMSUNG:AU - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Dishwashers:SAMSUNG:AU - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Vacuum Cleaners:SAMSUNG:AU - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Purifier:SAMSUNG:AU - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Conditioner:SAMSUNG:AU - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Oven:SAMSUNG:AU - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hob:SAMSUNG:AU - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.",...,"Vacuum Cleaners:AEG:AU - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Purifier:AEG:AU - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Conditioner:AEG:AU - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Oven:AEG:AU - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hob:AEG:AU - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Cooker:AEG:AU - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hood:AEG:AU - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Microwave:AEG:AU - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Fryers:AEG:AU - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","None of these:AEG:AU - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply."
0,78293,1.0,NaN,NaN,NaN,NaN,NaN,7.0,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,78361,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [61]:
# Unpivot the DataFrame, renaming 'Variable' to 'Product_Category'
unpivoted_AU = Australia.melt(id_vars=['Response ID'], var_name='Product_Category', value_name='Seen or Heard Y/N')

# Create a duplicate of the 'Product_Category' column named 'Product_Category_cleaned'
unpivoted_AU['Product_Category_cleaned'] = unpivoted_AU['Product_Category']

# Rearrange columns to place 'Product_Category_cleaned' next to 'Product_Category'
unpivoted_AU = unpivoted_AU[['Response ID', 'Product_Category', 'Product_Category_cleaned', 'Seen or Heard Y/N']]

# Remove the unwanted suffix from the 'Product_Category_cleaned' column only
unpivoted_AU['Product_Category_cleaned'] = unpivoted_AU['Product_Category_cleaned'].str.replace(
    ":AU - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.", 
    "", 
    regex=False
)

print(unpivoted_AU.tail())

       Response ID                                   Product_Category  \
66691        95482  None of these:AEG:AU - Which product categorie...   
66692        95486  None of these:AEG:AU - Which product categorie...   
66693        95487  None of these:AEG:AU - Which product categorie...   
66694        95489  None of these:AEG:AU - Which product categorie...   
66695        95490  None of these:AEG:AU - Which product categorie...   

      Product_Category_cleaned  Seen or Heard Y/N  
66691        None of these:AEG               99.0  
66692        None of these:AEG                NaN  
66693        None of these:AEG                NaN  
66694        None of these:AEG                NaN  
66695        None of these:AEG                NaN  


In [62]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Product_Category_cleaned'
for entry in unpivoted_AU['Product_Category_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_AU['Brand'] = brand_names
unpivoted_AU['Ad_Type'] = ad_types

In [63]:
count = unpivoted_AU['Seen or Heard Y/N'].value_counts()
print(count)

Seen or Heard Y/N
1.0     594
2.0     478
99.0    387
5.0     381
4.0     354
8.0     298
7.0     273
12.0    259
3.0     226
10.0    224
13.0    192
6.0     165
11.0    135
9.0     107
Name: count, dtype: int64


In [64]:
# Drop the original 'Product_Category_cleaned' column
unpivoted_AU = unpivoted_AU.drop(columns=['Product_Category_cleaned'])

# Rename 'Ad_Type' to 'Product_Category_cleaned'
unpivoted_AU = unpivoted_AU.rename(columns={'Ad_Type': 'Product_Category_cleaned'})

# Rearrange columns to place 'Product_Category_cleaned' next to 'Product_Category'
unpivoted_AU = unpivoted_AU[['Response ID', 'Product_Category', 'Product_Category_cleaned', 'Seen or Heard Y/N', 'Brand']]

# Replace values in the 'Seen or Heard Y/N' column
unpivoted_AU['Seen or Heard Y/N'] = np.where(unpivoted_AU['Seen or Heard Y/N'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]), 1, 
                        np.where(unpivoted_AU['Seen or Heard Y/N'] == 99, 99,  
                        np.where(unpivoted_AU['Seen or Heard Y/N'].isna(), 0, unpivoted_AU['Seen or Heard Y/N'])))

In [65]:
unpivoted_AU.head()

,Response ID,Product_Category,Product_Category_cleaned,Seen or Heard Y/N,Brand
0,78293,Refrigerator:SAMSUNG:AU - Which product catego...,Refrigerator,1.0,SAMSUNG
1,78361,Refrigerator:SAMSUNG:AU - Which product catego...,Refrigerator,0.0,SAMSUNG
2,78394,Refrigerator:SAMSUNG:AU - Which product catego...,Refrigerator,0.0,SAMSUNG
3,78406,Refrigerator:SAMSUNG:AU - Which product catego...,Refrigerator,0.0,SAMSUNG
4,78429,Refrigerator:SAMSUNG:AU - Which product catego...,Refrigerator,0.0,SAMSUNG


In [66]:
count = unpivoted_AU['Seen or Heard Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_AU['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_AU)
print(f"Total number of rows: {total_rows}")

Seen or Heard Y/N
0.0     62623
1.0      3686
99.0      387
Name: count, dtype: int64


Brand
SAMSUNG         5558
LG              5558
DYSON           5558
WESTINGHOUSE    5558
BOSCH           5558
HISENSE         5558
MIELE           5558
SMEG            5558
ELECTROLUX      5558
HAIER           5558
AEG             5558
Name: count, dtype: int64


Total number of rows: 66696


In [67]:
# Count blank (empty or NaN) values in Brand
blank_count = unpivoted_AU[unpivoted_AU['Brand'].isna() | (unpivoted_AU['Brand'] == '')].shape[0]
print("Count of blank values in Brand:", blank_count)

Count of blank values in Brand: 5558


In [68]:
# Replace NaN values in 'Brand' column with 'FISHER&PAYKEL'
unpivoted_AU['Brand'] = unpivoted_AU['Brand'].fillna('FISHER&PAYKEL')

# Check the count of NaN values after replacement
na_count = unpivoted_AU['Brand'].isna().sum()
print("Number of NaN values in Brand after replacement:", na_count)

Number of NaN values in Brand after replacement: 0


In [69]:
count = unpivoted_AU['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_AU)
print(f"Total number of rows: {total_rows}")

Brand
SAMSUNG          5558
LG               5558
DYSON            5558
WESTINGHOUSE     5558
FISHER&PAYKEL    5558
BOSCH            5558
HISENSE          5558
MIELE            5558
SMEG             5558
ELECTROLUX       5558
HAIER            5558
AEG              5558
Name: count, dtype: int64


Total number of rows: 66696


### Switzerland

In [70]:
Switzerland.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 398 entries, 0 to 397
Columns: 169 entries, Response ID to None of these:SCHULTHESS:SZ - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.
dtypes: float64(168), int64(1)
memory usage: 525.6 KB


In [71]:
Switzerland.head(2) #initial dataset

,Response ID,"Refrigerator:V ZUG:SZ - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Washing Machine:V ZUG:SZ - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Tumble Dryers:V ZUG:SZ - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Dishwashers:V ZUG:SZ - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Vacuum Cleaners:V ZUG:SZ - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Purifier:V ZUG:SZ - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Conditioner:V ZUG:SZ - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Oven:V ZUG:SZ - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hob:V ZUG:SZ - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.",...,"Vacuum Cleaners:SCHULTHESS:SZ - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Purifier:SCHULTHESS:SZ - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Conditioner:SCHULTHESS:SZ - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Oven:SCHULTHESS:SZ - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hob:SCHULTHESS:SZ - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Cooker:SCHULTHESS:SZ - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Hood:SCHULTHESS:SZ - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Microwave:SCHULTHESS:SZ - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","Air Fryers:SCHULTHESS:SZ - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.","None of these:SCHULTHESS:SZ - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply."
0,78375,NaN,2.0,3.0,4.0,NaN,NaN,NaN,8.0,9.0,...,5.0,NaN,NaN,NaN,NaN,NaN,NaN,12.0,13.0,NaN
1,78387,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [72]:
# Unpivot the DataFrame, renaming 'Variable' to 'Product_Category'
unpivoted_SZ = Switzerland.melt(id_vars=['Response ID'], var_name='Product_Category', value_name='Seen or Heard Y/N')

# Create a duplicate of the 'Product_Category' column named 'Product_Category_cleaned'
unpivoted_SZ['Product_Category_cleaned'] = unpivoted_SZ['Product_Category']

# Rearrange columns to place 'Product_Category_cleaned' next to 'Product_Category'
unpivoted_SZ = unpivoted_SZ[['Response ID', 'Product_Category', 'Product_Category_cleaned', 'Seen or Heard Y/N']]

# Remove the unwanted suffix from the 'Product_Category_cleaned' column only
unpivoted_SZ['Product_Category_cleaned'] = unpivoted_SZ['Product_Category_cleaned'].str.replace(
    ":SZ - Which product categories from each brand have you recently seen, heard, or read anything about? Select all that apply.", 
    "", 
    regex=False
)

print(unpivoted_SZ.tail())

       Response ID                                   Product_Category  \
66859        95956  None of these:SCHULTHESS:SZ - Which product ca...   
66860        95963  None of these:SCHULTHESS:SZ - Which product ca...   
66861        95971  None of these:SCHULTHESS:SZ - Which product ca...   
66862        95973  None of these:SCHULTHESS:SZ - Which product ca...   
66863        96100  None of these:SCHULTHESS:SZ - Which product ca...   

       Product_Category_cleaned  Seen or Heard Y/N  
66859  None of these:SCHULTHESS                NaN  
66860  None of these:SCHULTHESS                NaN  
66861  None of these:SCHULTHESS                NaN  
66862  None of these:SCHULTHESS                NaN  
66863  None of these:SCHULTHESS                NaN  


In [73]:
# Initialize lists to store processed data
brand_names = []
ad_types = []

# Process each item in 'Product_Category_cleaned'
for entry in unpivoted_SZ['Product_Category_cleaned']:
    # Extract brand name (e.g., Samsung, LG, etc.)
    brand = re.search(r":([A-Za-z]+)$", entry)  # Adjusted to capture brand names at the end of the string
    if brand:
        brand_name = brand.group(1)
    else:
        brand_name = None
    
    # Extract type of advertising (e.g., Large Roadside Posters, On bus-stops, etc.)
    ad_type = re.search(r"^[^:]+", entry)  # Capture everything before the first colon
    if ad_type:
        ad_type_name = ad_type.group(0).strip()
    else:
        ad_type_name = None

    # Append to lists
    brand_names.append(brand_name)
    ad_types.append(ad_type_name)

# Add extracted information to the DataFrame
unpivoted_SZ['Brand'] = brand_names
unpivoted_SZ['Ad_Type'] = ad_types

In [74]:
count = unpivoted_SZ['Seen or Heard Y/N'].value_counts()
print(count)

Seen or Heard Y/N
2.0     446
5.0     430
1.0     408
8.0     402
4.0     358
3.0     326
10.0    307
9.0     291
99.0    261
12.0    259
6.0     212
7.0     203
13.0    198
11.0    190
Name: count, dtype: int64


In [75]:
# Drop the original 'Product_Category_cleaned' column
unpivoted_SZ = unpivoted_SZ.drop(columns=['Product_Category_cleaned'])

# Rename 'Ad_Type' to 'Product_Category_cleaned'
unpivoted_SZ = unpivoted_SZ.rename(columns={'Ad_Type': 'Product_Category_cleaned'})

# Rearrange columns to place 'Product_Category_cleaned' next to 'Product_Category'
unpivoted_SZ = unpivoted_SZ[['Response ID', 'Product_Category', 'Product_Category_cleaned', 'Seen or Heard Y/N', 'Brand']]

# Replace values in the 'Seen or Heard Y/N' column
unpivoted_SZ['Seen or Heard Y/N'] = np.where(unpivoted_SZ['Seen or Heard Y/N'].isin([1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13]), 1, 
                        np.where(unpivoted_SZ['Seen or Heard Y/N'] == 99, 99,  
                        np.where(unpivoted_SZ['Seen or Heard Y/N'].isna(), 0, unpivoted_SZ['Seen or Heard Y/N'])))

In [76]:
unpivoted_SZ.head()

,Response ID,Product_Category,Product_Category_cleaned,Seen or Heard Y/N,Brand
0,78375,Refrigerator:V ZUG:SZ - Which product categori...,Refrigerator,0.0,None
1,78387,Refrigerator:V ZUG:SZ - Which product categori...,Refrigerator,0.0,None
2,78581,Refrigerator:V ZUG:SZ - Which product categori...,Refrigerator,0.0,None
3,78735,Refrigerator:V ZUG:SZ - Which product categori...,Refrigerator,0.0,None
4,78849,Refrigerator:V ZUG:SZ - Which product categori...,Refrigerator,0.0,None


In [77]:
count = unpivoted_SZ['Seen or Heard Y/N'].value_counts()
print(count)

print("\n")

count = unpivoted_SZ['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_SZ)
print(f"Total number of rows: {total_rows}")

Seen or Heard Y/N
0.0     62573
1.0      4030
99.0      261
Name: count, dtype: int64


Brand
ELECTROLUX    5572
MIELE         5572
DYSON         5572
BOSCH         5572
SAMSUNG       5572
SIEMENS       5572
PHILIPS       5572
BAUKNECHT     5572
AEG           5572
LIEBHERR      5572
SCHULTHESS    5572
Name: count, dtype: int64


Total number of rows: 66864


In [78]:
# Count blank (empty or NaN) values in Brand
blank_count = unpivoted_SZ[unpivoted_SZ['Brand'].isna() | (unpivoted_SZ['Brand'] == '')].shape[0]
print("Count of blank values in 'Brand:", blank_count)

Count of blank values in 'Brand: 5572


In [79]:
# Replace NaN values in 'Brand' column with 'V Zug'
unpivoted_SZ['Brand'] = unpivoted_SZ['Brand'].fillna('V ZUG')

# Check the count of NaN values after replacement
na_count = unpivoted_SZ['Brand'].isna().sum()
print("Number of NaN values in Brand after replacement:", na_count)

Number of NaN values in Brand after replacement: 0


In [80]:
count = unpivoted_SZ['Brand'].value_counts()
print(count)

print("\n")

total_rows = len(unpivoted_SZ)
print(f"Total number of rows: {total_rows}")

Brand
V ZUG         5572
ELECTROLUX    5572
MIELE         5572
DYSON         5572
BOSCH         5572
SAMSUNG       5572
SIEMENS       5572
PHILIPS       5572
BAUKNECHT     5572
AEG           5572
LIEBHERR      5572
SCHULTHESS    5572
Name: count, dtype: int64


Total number of rows: 66864


### Concat and save them in another csv file

In [81]:
# Concatenate the melted DataFrames
combined_q18 = pd.concat([unpivoted_GM, unpivoted_SW, unpivoted_TH, unpivoted_IT, unpivoted_PO, 
                                unpivoted_SZ, unpivoted_VN, unpivoted_AU, unpivoted_UK], ignore_index=True)


# Define conditions for categorization
conditions = [
    combined_q18['Product_Category_cleaned'].isin(["Washing Machine", "Tumble Dryers", "Dishwashers"]),
    combined_q18['Product_Category_cleaned'].isin(["Refrigerator", "Oven", "Hob", "Cooker", "Hood", "Microwave"]),
    combined_q18['Product_Category_cleaned'].isin(["Vacuum Cleaners", "Air Purifier", "Air Conditioner", "Air Fryers"])
]

# Define corresponding outputs for the conditions
outputs = ["Care", "Taste", "Wellbeing & SDA"]

# Assign the new column 'Experience Area' based on conditions
combined_q18['Experience_Area'] = np.select(conditions, outputs, default="Other")

# Display the result
print(combined_q18.head())

# combined_q18.dropna(subset=['Response ID'], inplace=True)

combined_q18 = combined_q18.sort_values(by='Response ID', ascending=True)

# combined_q18.to_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\PROCESSED-DATA\SEPTEMBER\combined_q18.xlsx', float_format='%.0f', index=False)
# combined_q18.to_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\PROCESSED-DATA\OCTOBER\combined_q18.xlsx', float_format='%.0f', index=False)
# combined_q18.to_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\PROCESSED-DATA\NOVEMBER\combined_q18.xlsx', float_format='%.0f', index=False)
combined_q18.to_excel(r'C:\Users\aisar\OneDrive\Documents\ELECTROLUX\PROCESSED-DATA\DECEMBER\combined_q18.xlsx', float_format='%.0f', index=False)

print(combined_q18.head(2))

   Response ID                                   Product_Category  \
0        78230  Refrigerator:BOSCH:GM - Which product categori...   
1        78263  Refrigerator:BOSCH:GM - Which product categori...   
2        78268  Refrigerator:BOSCH:GM - Which product categori...   
3        78278  Refrigerator:BOSCH:GM - Which product categori...   
4        78299  Refrigerator:BOSCH:GM - Which product categori...   

  Product_Category_cleaned  Seen or Heard Y/N  Brand Experience_Area  
0             Refrigerator                0.0  BOSCH           Taste  
1             Refrigerator                0.0  BOSCH           Taste  
2             Refrigerator                0.0  BOSCH           Taste  
3             Refrigerator                0.0  BOSCH           Taste  
4             Refrigerator                0.0  BOSCH           Taste  
       Response ID                                   Product_Category  \
13860        78230  Oven:SIEMENS:GM - Which product categories fro...   
38412        

In [82]:
# Count blank (NaN) rows in the 'Response ID' column
blank_count = combined_q18['Response ID'].isna().sum()
print(f"Number of blank rows in 'Response ID': {blank_count}")

Number of blank rows in 'Response ID': 0


In [83]:
combined_q18.info()

<class 'pandas.core.frame.DataFrame'>
Index: 602112 entries, 13860 to 339201
Data columns (total 6 columns):
 #   Column                    Non-Null Count   Dtype  
---  ------                    --------------   -----  
 0   Response ID               602112 non-null  int64  
 1   Product_Category          602112 non-null  object 
 2   Product_Category_cleaned  602112 non-null  object 
 3   Seen or Heard Y/N         602112 non-null  float64
 4   Brand                     602112 non-null  object 
 5   Experience_Area           602112 non-null  object 
dtypes: float64(1), int64(1), object(4)
memory usage: 32.2+ MB


In [84]:
count = combined_q18['Experience_Area'].value_counts()
print(count)

Experience_Area
Taste              258048
Wellbeing & SDA    172032
Care               129024
Other               43008
Name: count, dtype: int64
